In [1]:
!nvidia-smi

Sun Jul 16 08:48:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3D:00.0 Off |                    0 |
| N/A   35C    P0    42W / 300W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from pynvml import *
import torch.optim as optim


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()
    


In [3]:
print_gpu_utilization()

GPU memory occupied: 3965 MB.


In [4]:
!pip install -q datasets

In [6]:
model_name = "t5-small"
from transformers import AutoTokenizer, T5ForConditionalGeneration

False
'CUDASetup' object has no attribute 'cuda'


/home/toibazd/miniconda3/lib/python3.9/site-packages/bitsandbytes-0.40.0.post4-py3.9.egg/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/home/toibazd/miniconda3/lib/python3.9/site-packages/bitsandbytes-0.40.0.post4-py3.9.egg/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/toibazd/miniconda3 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/toibazd/miniconda3/lib/python3.9/site-packages/bitsandbytes-0.40.0.post4-py3.9.egg/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('CentOS/7.9.2009')}
  warn(msg)


[2023-07-12 10:46:56,476] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-07-12 10:46:58.147883: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-12 10:46:58.355677: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-12 10:46:58.417342: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-12 10:47:00.424774: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


In [8]:
from datasets import load_dataset

In [9]:
train_data = load_dataset("wikisql", split="train+validation")
test_data = load_dataset("wikisql", split = "test")



Found cached dataset wikisql (/home/toibazd/.cache/huggingface/datasets/wikisql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d)
Found cached dataset wikisql (/home/toibazd/.cache/huggingface/datasets/wikisql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d)


In [10]:
def format_dataset(dataset_batch):
    return {"input":"translate to SQL: " + dataset_batch["question"],"target":dataset_batch["sql"]["human_readable"]}

train_data = train_data.map(format_dataset, remove_columns=train_data.column_names)

Loading cached processed dataset at /home/toibazd/.cache/huggingface/datasets/wikisql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d/cache-7cc98b0d68adc614.arrow


In [11]:
train_data[-1]

{'input': 'translate to SQL: What is the lifespan of the democratic party in New York, for which Terence J. Quinn is a representative?',
 'target': 'SELECT Lifespan FROM table WHERE Party = democratic AND State = new york AND Representative = terence j. quinn'}

In [12]:
type(train_data)

datasets.arrow_dataset.Dataset

In [13]:
test_data = test_data.map(format_dataset,remove_columns=test_data.column_names)

Loading cached processed dataset at /home/toibazd/.cache/huggingface/datasets/wikisql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d/cache-cd9f2e661c310a64.arrow


In [15]:
def map_to_lenght(x):
    x["input_len"] = len(tokenizer(x["input"]).input_ids)
    x["input_longer_256"] = int(x["input_len"]>256)
    x["input_longer_128"] = int(x["input_len"]>128)
    x["input_longet_64"] = int(x["input_len"]>64)
    x["output_len"] = len(tokenizer(x["target"]).input_ids)
    x["output_longet_256"] = int(x["output_len"]>256)
    x["output_longet_128"] = int(x["output_len"]>128)
    x["output_longet_64"] = int(x["output_len"]>64)
    return x

sample_size = 10000
data_stats = train_data.select(range(sample_size)).map(map_to_lenght, num_proc=4)

    

Loading cached processed dataset at /home/toibazd/.cache/huggingface/datasets/wikisql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d/cache-6d13e08a1fa873ed_*_of_00004.arrow


In [16]:
data_stats[0]


{'input': 'translate to SQL: Tell me what the notes are for South Australia ',
 'target': 'SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA',
 'input_len': 14,
 'input_longer_256': 0,
 'input_longer_128': 0,
 'input_longet_64': 0,
 'output_len': 22,
 'output_longet_256': 0,
 'output_longet_128': 0,
 'output_longet_64': 0}

In [17]:
def compute_and_print(x):
    if len(x["input_len"])==sample_size:
        print(
            f"Input mean: {sum(x['input_len'])/sample_size} \n % of input len > 256: {sum(x['input_longer_256'])/sample_size}, \n % of input len > 128: {sum(x['input_longer_128'])/sample_size}, \n % of input len > 64: {sum(x['input_longet_64'])/sample_size}, \n Ouput mean: {sum(x['output_len'])/sample_size},\n% of output len > 256: {sum(x['output_longet_256'])/sample_size}, \n% of output len > 128: {sum(x['output_longet_128'])/sample_size}, \n% of output len > 64: {sum(x['output_longet_64'])/sample_size}")

output = data_stats.map(compute_and_print, batched=True, batch_size=-1)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Input mean: 19.8508 
 % of input len > 256: 0.0, 
 % of input len > 128: 0.0, 
 % of input len > 64: 0.0002, 
 Ouput mean: 20.0403,
% of output len > 256: 0.0, 
% of output len > 128: 0.0002, 
% of output len > 64: 0.0005


In [18]:
train_data

Dataset({
    features: ['input', 'target'],
    num_rows: 64776
})

In [19]:
def convert_to_features(example_batch, padding = "max_length"):
    inputs = tokenizer.batch_encode_plus(example_batch["input"], max_length=64, padding="max_length", truncation=True)
    targets = tokenizer.batch_encode_plus(example_batch["target"], max_length=64, padding="max_length",truncation = True)
    
    if padding == "max_length":
        targets["inputs_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in target] for target in targets["input_ids"]
        ]
    
    inputs["labels"] = targets['input_ids']
    return inputs

train_data = train_data.map(convert_to_features, batched=True, remove_columns=train_data.column_names)
test_data = test_data.map(convert_to_features, batched=True, remove_columns=test_data.column_names)

Map:   0%|          | 0/64776 [00:00<?, ? examples/s]

Map:   0%|          | 0/15878 [00:00<?, ? examples/s]

In [20]:
columns = ['input_ids', 'attention_mask', 'labels']

train_data.set_format(type='torch', columns=columns)
test_data.set_format(type='torch', columns=columns)

In [21]:
len(train_data["input_ids"][12330])


64

In [22]:
def sequence_check(list_of_seq):
    len_seq = len(list_of_seq[0])
    for seq in list_of_seq:
        if len(seq)!=len_seq:
            print(len(seq))
            return False
        
    return True

In [23]:
sequence_check(train_data['input_ids'])

True

In [25]:
len(train_data["labels"][0])


64

In [26]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments


In [27]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-small-finetuned-wikisql",
    per_device_train_batch_size=16,
    num_train_epochs=10,
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    logging_steps=500,
    save_strategy="epoch",
    #save_steps=1000,
    #eval_steps=1000,
    overwrite_output_dir=True,
    save_total_limit=3,
    load_best_model_at_end=True,
    push_to_hub=False
    #fp16=True 
)

In [28]:
! pip install -q rouge_score


In [34]:
from datasets import load_metric
rouge = load_metric("rouge")

/tmp/ipykernel_23353/2398894310.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


In [35]:
def compute_metrics(pred):
    label_ids = pred.label_ids
    pred_ids = pred.predictions
    
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids== -101] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid
    print_gpu_utilization()
    return {
        "rouge2 precision": round(rouge_output.precision, 4),
        "rouge2 recall": round(rouge_output.recall, 4),
        "rouge2 F1 score": round(rouge_output.fmeasure, 4)
    }

In [36]:
test_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
    num_rows: 15878
})

In [37]:
trainer = Seq2SeqTrainer(
    model = model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset= test_data
)

In [38]:
trainer.evaluate()

RuntimeError: output with shape [16, 8, 1, 1] doesn't match the broadcast shape [16, 8, 1, 64]

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('/content/t5-small-finetuned-wikisql')

In [ ]:
tokenizer.save_pretrained('/content/t5-small-finetuned-wikisql')

In [ ]:
!nvidia-smi